In [31]:
import json
import struct
import numpy as np
import matplotlib.pyplot as plt
import photonforge as pf
import siepic_forge as siepic
import luxtelligence_lnoi400_forge as lxt
import tidy3d as td

td.config.logging_level = "ERROR"

# Set up technologies
siepic_tech = siepic.ebeam()
basic_tech = pf.basic_technology()
lxt_tech = lxt.lnoi400()
pf.config.default_technology = siepic_tech

# Initialize live viewer for real-time visualization
from photonforge.live_viewer import LiveViewer
viewer = LiveViewer()

# Define simulation parameters
wavelengths = np.linspace(1.53, 1.57, 101)
freqs = pf.C_0 / wavelengths

LiveViewer started at http://localhost:59660


In [32]:
dual_mode_spec = siepic_tech.ports["TE_1550_500"].copy()
dual_mode_spec.num_modes = 2  # Use both modes

siepic_tech.add_port("TE-TM_1550_500", dual_mode_spec)
siepic_tech.ports["TE-TM_1550_500"]

PortSpec(description="Strip TE 1550 nm, w=500 nm", width=2, limits=(-1, 1.22), num_modes=2, added_solver_modes=0, polarization="", target_neff=3.5, default_radius=0, path_profiles=[(0.5, 0, (1, 0))])

In [33]:
plasmonic_gold_layer = pf.LayerSpec(layer=(15, 0), description="Metal", color="F5BC00", pattern="xx")

siepic_tech.add_layer("Plasmonic Gold", plasmonic_gold_layer)

Name,Layer,Description,Color,Pattern
Si,"(1, 0)",Waveguides,#ff80a818,\\
PinRec,"(1, 10)",SiEPIC,#00408018,/
PinRecM,"(1, 11)",SiEPIC,#00408018,/
Si_Litho193nm,"(1, 69)",Waveguides,#cc80a818,\
Waveguide,"(1, 99)",Waveguides,#ff80a818,\
Si slab,"(2, 0)",Waveguides,#80a8ff18,/
SiN,"(4, 0)",Waveguides,#a6cee318,\\
Oxide open (to BOX),"(6, 0)",Waveguides,#ffae0018,\
Text,"(10, 0)",,#0000ff18,\
M1_heater,"(11, 0)",Metal,#ebc63418,xx


In [34]:
plasmonic_gold_gap = 0.03  # Gap between waveguide and gold
plasmonic_gold_thickness = 0.14  # Thickness of the gold layer
wg_height = 0.22

plasmonic_gold_extrusion = pf.ExtrusionSpec(mask_spec=pf.MaskSpec((15, 0)), 
                                            medium={"optical": td.material_library['Au']['JohnsonChristy1972'], 
                                                    "electrical": td.LossyMetalMedium(conductivity=17.0, fit_param={'attrs': {}, 'max_num_poles': 16, 'tolerance_rms': 0.001, 'frequency_sampling_points': 20, 'log_sampling': True, 'type': 'SurfaceImpedanceFitterParam'}, frequency_range=(100000000.0, 200000000000.0))}, 
                                                    limits=(wg_height+plasmonic_gold_gap, wg_height+plasmonic_gold_gap+plasmonic_gold_thickness), sidewall_angle=0, reference="top")
siepic_tech.insert_extrusion_spec(4, plasmonic_gold_extrusion)

Name,Layer,Description,Color,Pattern
Si,"(1, 0)",Waveguides,#ff80a818,\\
PinRec,"(1, 10)",SiEPIC,#00408018,/
PinRecM,"(1, 11)",SiEPIC,#00408018,/
Si_Litho193nm,"(1, 69)",Waveguides,#cc80a818,\
Waveguide,"(1, 99)",Waveguides,#ff80a818,\
Si slab,"(2, 0)",Waveguides,#80a8ff18,/
SiN,"(4, 0)",Waveguides,#a6cee318,\\
Oxide open (to BOX),"(6, 0)",Waveguides,#ffae0018,\
Text,"(10, 0)",,#0000ff18,\
M1_heater,"(11, 0)",Metal,#ebc63418,xx


In [35]:
class ThermalModel(pf.Model):
    def __init__(self, n_complex, voltage=0, coefficient=3e-4):
        super().__init__(
            n_complex=n_complex,
            voltage=voltage,
            coefficient=coefficient,
        )
        self.n_complex = np.array(n_complex, ndmin=2)
        self.voltage = voltage
        self.coefficient = coefficient

    def __copy__(self):
        return ThermalModel(self.n_complex, self.voltage, self.coefficient)

    def __deepcopy__(self, memo=None):
        # n_complex is an array, so we want to make sure to create a deep copy of it.
        # Other values (voltage and coefficient) are immutable (floats), so we can use them directly.
        return ThermalModel(self.n_complex.copy(), self.voltage, self.coefficient)

    def __repr__(self):
        return f"ThermalModel({self.n_complex!r}, {self.voltage!r}, {self.coefficient!r})"

    def __str__(self):
        return f"ThermalModel at {self.voltage} V"

    @property
    def as_bytes(self):
        coeffs = struct.pack("<2d", self.voltage, self.coefficient)
        shape = struct.pack("<2l", *self.n_complex.shape)
        n_data = self.n_complex.astype(complex).tobytes()
        # Add version 0 as first byte
        return b"\x00" + coeffs + shape + n_data

    @classmethod
    def from_bytes(cls, byte_repr):
        version = byte_repr[0]
        if version != 0:
            raise RuntimeError(f"Incompatible version for ThermalModel: {version}")

        byte_repr = byte_repr[1:]
        fmt = "<2d2l"
        head_len = struct.calcsize(fmt)
        voltage, coefficient, rows, cols = struct.unpack(fmt, byte_repr[:head_len])

        byte_repr = byte_repr[head_len:]
        n_complex = np.frombuffer(byte_repr, dtype=complex).reshape((rows, cols))

        return cls(n_complex, voltage, coefficient)

    @pf.cache_s_matrix
    def start(self, component, frequencies, voltage=None, **kwargs):
        # Allow overriding voltage as an `s_matrix` kwarg too
        if voltage is None:
            voltage = self.voltage
        n_complex = self.n_complex + self.coefficient * voltage**2
        wg_model = pf.WaveguideModel(n_complex)
        return wg_model.start(component, frequencies, **kwargs)


pf.register_model_class(ThermalModel)

In [36]:
# Define the width of the regular silicon strip waveguide (µm)
w_strip = 0.5

# Define the width of the subwavelength grating (SWG) equivalent waveguide (µm)
w_swg = 0.555

# Create a port specification for the strip waveguide
port_spec_strip = pf.PortSpec(
    description="Multi mode strip",  # descriptive label for identification
    width=2.5,  # total port width (simulation window)
    num_modes=2,  # number of supported optical modes
    target_neff=3.5,  # target effective index for solver initialization
    limits=(-1.5, 1.22),  # vertical boundaries of the simulation domain
    path_profiles=[
        (w_strip, 0, (1, 0))
    ],  # waveguide path geometry (width, offset, direction)
)

# Create a port specification for the SWG (subwavelength) waveguide
port_spec_swg = pf.PortSpec(
    description="Multi mode swg",
    width=2.5,
    num_modes=2,
    target_neff=3.5,
    limits=(-1.5, 1.22),
    path_profiles=[(w_swg, 0, (1, 0))],
)

In [37]:
# Define a parametric component for the SWG-assisted PBS
@pf.parametric_component
def create_pbs(
    *,
    spec_strip="TE-TM_1550_500",
    spec_swg=port_spec_swg,
    w_corrugation=0.130,
    gap=0.200,
    swg_period=0.240,
    swg_duty_cycle=0.5,
    n_periods=30,
    s_bend_length=6,
    s_bend_offset=2,
):
    # Resolve string-based specs from the default technology if needed
    if isinstance(spec_strip, str):
        spec_strip = pf.config.default_technology.ports[spec_strip]
    if isinstance(spec_swg, str):
        spec_swg = pf.config.default_technology.ports[spec_swg]

    # Extract effective silicon widths from the provided PortSpecs
    w_strip, _ = spec_strip.path_profile_for("Si")
    w_swg, _ = spec_swg.path_profile_for("Si")

    # Compute total coupling length from SWG period and number of periods
    coupling_length = n_periods * swg_period

    # Create one SWG unit cell component consisting of ridge + groove
    unit_cell = pf.Component("SWG Period")

    # Build the ridge and groove rectangles for one period
    ridge_length = swg_duty_cycle * swg_period
    ridge = pf.Rectangle((0, -w_swg / 2), (ridge_length, w_swg / 2))
    groove = pf.Rectangle(
        (ridge_length, -w_swg / 2), (swg_period, w_swg / 2 - w_corrugation)
    )

    # Add ridge and groove into the unit cell along +x
    unit_cell.add((1, 0), ridge, groove)

    # Create the parent PBS component
    pbs = pf.Component("PBS")

    # Instantiate the SWG array by repeating the unit cell N times
    swg_array = pf.Reference(
        component=unit_cell, columns=n_periods, rows=1, spacing=(swg_period, 0.0)
    )

    # Add a short straight continuation for the SWG waveguide after coupling region
    # swg_wg = pf.Rectangle(
    #     (coupling_length, -w_swg / 2),
    #     (coupling_length + s_bend_length, w_swg / 2),
    # )

    # Route the strip waveguide: lead-in, straight through coupler, then S-bend
    strip_wg = (
        pf.Path(origin=(-1.0, gap + (w_swg + w_strip) / 2), width=w_strip)
        .segment(endpoint=(coupling_length + 1.0, 0), relative=True)
        .s_bend(endpoint=(s_bend_length, s_bend_offset), relative=True)
    )

    swg_taper = (
        pf.Path(origin=(coupling_length, 0), width=w_swg)
        .segment(endpoint=(s_bend_length, 0), relative=True, width=w_strip)
    )

    # Add geometry to the PBS component (placed along +x)
    pbs.add((1, 0), swg_array, strip_wg, swg_taper)

    # Detect and add ports for strip (input) and SWG (output on +x boundary)
    pbs.add_port(pbs.detect_ports([spec_strip]))
    # pbs.add_port(pbs.detect_ports([spec_swg], on_boundary="+x"))

    # Attach a Tidy3D model for simulation
    pbs.add_model(
        pf.Tidy3DModel(
            monitors=[
                td.FieldMonitor(
                    name="field",
                    center=(0, 0, 0.11),
                    size=(td.inf, td.inf, 0),
                    freqs=[pf.C_0 / 1.55],
                )
            ],
        )
    )
    return pbs


# Instantiate and view the PBS component with default parameters
# pbs = create_pbs()
# viewer(pbs)

c:\Users\irfan\AppData\Local\Programs\Python\Python313\Lib\site-packages\photonforge\parametric_utils.py:208: RuntimeWarning: Component function '__main__.create_pbs' previously registered will be overwritten.
  return _decorator(decorated_function)


In [38]:
@pf.parametric_component
def create_mzm(*, name, port_spec="TE_1550_500", coupling_distance=0.6, coupling_length=5.35, ps_length=100):

    # Bend and Straight waveguide to form the microring
    phase_shifter = pf.parametric.straight(name=f"ps", port_spec=port_spec, length=ps_length)
    straight = pf.parametric.straight(port_spec=port_spec, length=ps_length)
    bend = pf.parametric.bend(port_spec=port_spec, radius=5)
    coupler = pf.parametric.dual_ring_coupler(
        port_spec=port_spec, coupling_distance=coupling_distance, coupling_length=coupling_length, radius=5, 
        tidy3d_model_kwargs={
            # These symmetries are not correct for arbitrary ports.
            # They can be used in this case for the fundamental mode.
            "port_symmetries": [
                ("P1", "P0", "P3", "P2"),
                ("P2", "P3", "P0", "P1"),
                ("P3", "P2", "P1", "P0"),
            ],
        },
    )

    # Solve for the port mode of the waveguide and extract the complex refractive index
    alpha = 10
    kappa = (alpha * wavelengths * 1e-4 * np.log(10)) / (40 * np.pi)
    mode_solver = pf.port_modes(port=phase_shifter.ports["P0"], frequencies=freqs)
    n_complex = mode_solver.data.n_complex.values.T + 1j * kappa  # add propagation loss

    thermal_model = ThermalModel(n_complex=n_complex)
    phase_shifter.add_model(thermal_model, "Thermal")

    mzm = pf.Component(name)

    # form main microring structure
    cp1_ref = mzm.add_reference(coupler)
    cp2_ref = mzm.add_reference(coupler)
    ps1_ref = mzm.add_reference(phase_shifter)
    bend1_ref = mzm.add_reference(bend)
    bend2_ref = mzm.add_reference(bend) 
    bend3_ref = mzm.add_reference(bend)
    bend4_ref = mzm.add_reference(bend) 
    str1_ref = mzm.add_reference(straight)

    #Coupler Kiri
    bend1_ref.connect("P1", cp1_ref["P3"])
    bend2_ref.connect("P0", cp1_ref["P2"])
    #Tengah
    # trans1_ref.connect("P0", bend1_ref["P0"])
    ps1_ref.connect("P0", bend1_ref["P0"])
    str1_ref.connect("P0", bend2_ref["P1"])

    # trans2_ref.connect("P1", ps1_ref["P1"])
    bend3_ref.connect("P1", ps1_ref["P1"])
    bend4_ref.connect("P0", str1_ref["P1"])
    #Coupler Kanan
    cp2_ref.connect("P1", bend3_ref["P0"])
    cp2_ref.connect("P0", bend4_ref["P1"])


    # Define the heater and electrical path
    terminal_width = 10
    heater_width = 2

    heater = (
        pf.Path((ps1_ref.x_min, ps1_ref.y_mid), heater_width)
        .segment((ps1_ref.x_max, ps1_ref.y_mid), heater_width)
    )

    route_vp = (
        pf.Path((cp1_ref.x_mid-terminal_width/2, ps1_ref.y_mid), terminal_width)
        .segment((cp1_ref.x_mid+terminal_width/2, ps1_ref.y_mid), terminal_width)
        .segment((ps1_ref.x_min, ps1_ref.y_mid), heater_width)
    )

    route_vn = (
        pf.Path((ps1_ref.x_max, ps1_ref.y_mid), heater_width)
        .segment((cp2_ref.x_mid-terminal_width/2, ps1_ref.y_mid), terminal_width)
        .segment((cp2_ref.x_mid+terminal_width/2, ps1_ref.y_mid), terminal_width)
    )

    mzm.add((11,0), heater)
    mzm.add((12,0), route_vp)
    mzm.add((12,0), route_vn)
    mzm.add_terminal(pf.Terminal((12,0), pf.Rectangle(size=(terminal_width, terminal_width), center=(cp1_ref.x_mid, ps1_ref.y_mid))), "VP")
    mzm.add_terminal(pf.Terminal((12,0), pf.Rectangle(size=(terminal_width, terminal_width), center=(cp2_ref.x_mid, ps1_ref.y_mid))), "VN")

    ## COMPONENT MODELS
    mzm.add_port(mzm.detect_ports([port_spec]))
    mzm.add_model(pf.CircuitModel(), "CircuitModel")

    port_symmetries = [
        ("P1", "P0", "P3", "P2"),
        ("P2", "P3", "P0", "P1"),
        ("P3", "P2", "P1", "P0"),
    ]

    field_monitor = td.FieldMonitor(
        center=(0, 0, 0.11), size=(td.inf, td.inf, 0), freqs=[freqs.mean()], name="field"
    )

    # mzm.add_model(pf.Tidy3DModel(port_symmetries=port_symmetries, monitors=[field_monitor]), "Tidy3DModel")

    return mzm

# mzm = create_mzm(name="mzm", port_spec="TE-TM_1550_500", coupling_length=4.3)
# viewer(mzm)

c:\Users\irfan\AppData\Local\Programs\Python\Python313\Lib\site-packages\photonforge\parametric_utils.py:208: RuntimeWarning: Component function '__main__.create_mzm' previously registered will be overwritten.
  return _decorator(decorated_function)


In [39]:
@pf.parametric_component
def create_hybrid_rotator(*, gold_length=4.25, gold_width=0.252, gold_location="Left"):

    wg_length = 8.5
    wg_height = 0.22
    wg_width = 0.5

    # Create an empty component named "HybridRotator"
    hybrid_rotator = pf.Component("HybridRotator")

    waveguide = pf.Rectangle(size=(wg_length, wg_width))
    gold_plate = pf.Rectangle(size=(gold_length, gold_width), center=(0, (-1 if gold_location == "Right" else 1)*(wg_width/2-gold_width/2)))

    # Add the geometry to the "Si" and "Plasmonic Gold" layers
    hybrid_rotator.add("Si", waveguide)
    hybrid_rotator.add("Plasmonic Gold", gold_plate)

    hybrid_rotator.add_port(
        pf.Port(center=(-wg_length/2, 0), input_direction=0, spec=siepic_tech.ports["TE_1550_500"])
    )

    hybrid_rotator.add_port(
        pf.Port(center=(wg_length/2, 0), input_direction=180, spec=siepic_tech.ports["TE-TM_1550_500"])
    )

    field_monitor = td.FieldMonitor(
        center=(0, 0, 0.11), size=(td.inf, td.inf, 0), freqs=[freqs.mean()], name="field"
    )

    input_monitor = td.FieldMonitor(
        center=[-wg_length/2, 0.0, wg_height / 2],
        size=[0.0, wg_width * 3, wg_height * 3],
        freqs=freqs,
        name="input",
        fields=["Ex", "Ey", "Ez", "Hx", "Hy", "Hz"]
    )

    output_monitor = td.FieldMonitor(
        center=[wg_length/2, 0.0, wg_height / 2],
        size=[0.0, wg_width * 3, wg_height * 3],
        freqs=freqs,
        name="output",
        fields=["Ex", "Ey", "Ez", "Hx", "Hy", "Hz"]
    )

    # Include the Tidy3D simulation model
    hybrid_rotator.add_model(pf.Tidy3DModel(monitors=[field_monitor, input_monitor, output_monitor]), "Tidy3DModel")
    return hybrid_rotator


# Instantiate the component with custom dimensions
# hybrid_rotator = create_hybrid_rotator(gold_location="Right")
# viewer(hybrid_rotator)

In [51]:
bob_top = pf.Component("bob_top")

straight = pf.parametric.straight(port_spec="TE-TM_1550_500", length=140)
half_straight = pf.parametric.straight(port_spec="TE-TM_1550_500", length=50)
quarter_straight = pf.parametric.straight(port_spec="TE-TM_1550_500", length=10)
bend = pf.parametric.bend(port_spec="TE-TM_1550_500", radius=5, euler_fraction=0.5, angle=90)

grating_coupler = siepic.component("ebeam_gc_te1550")

pbs = create_pbs()
mzm_base = create_mzm(name="mzm_base", port_spec="TE-TM_1550_500", coupling_length=4.3)
rotator_45 = create_hybrid_rotator(gold_location="Left")
rotator_n45 = create_hybrid_rotator(gold_location="Right")
rotator_0 = pf.parametric.transition(port_spec1="TE-TM_1550_500", port_spec2="TE-TM_1550_500", length=8.5)

gc1_ref = bob_top.add_reference(grating_coupler)
gc2_ref = bob_top.add_reference(grating_coupler)
gc3_ref = bob_top.add_reference(grating_coupler)

MZI1_ref = bob_top.add_reference(mzm_base)
MZI2_ref = bob_top.add_reference(mzm_base)

# st1_ref = bob_top.add_reference(straight)
# st2_ref = bob_top.add_reference(straight)
# st3_ref = bob_top.add_reference(straight)
# st4_ref = bob_top.add_reference(straight)

# hst1_ref = bob_top.add_reference(half_straight)
# hst2_ref = bob_top.add_reference(half_straight)
# hst3_ref = bob_top.add_reference(half_straight)
# hst4_ref = bob_top.add_reference(half_straight)
# hst5_ref = bob_top.add_reference(half_straight)
# hst6_ref = bob_top.add_reference(half_straight)
# hst7_ref = bob_top.add_reference(half_straight)
# hst8_ref = bob_top.add_reference(half_straight)

qst1_ref = bob_top.add_reference(quarter_straight)
qst2_ref = bob_top.add_reference(quarter_straight)
qst3_ref = bob_top.add_reference(quarter_straight)

bend1_ref = bob_top.add_reference(bend)
bend2_ref = bob_top.add_reference(bend)
bend3_ref = bob_top.add_reference(bend)
bend4_ref = bob_top.add_reference(bend)
bend5_ref = bob_top.add_reference(bend)
bend6_ref = bob_top.add_reference(bend)

pbs_ref = bob_top.add_reference(pbs)

rot1_ref = bob_top.add_reference(rotator_45)
rot2_ref = bob_top.add_reference(rotator_0)

#Grating Coupler, MZM input 
qst1_ref.connect("P0",gc1_ref["P0"])
bend1_ref.connect("P1",qst1_ref["P1"])
MZI1_ref.connect("P1",bend1_ref["P0"])

#Jalur Atas
bend2_ref.connect("P1",MZI1_ref["P3"])
rot1_ref.connect("P0",bend2_ref["P0"])
bend4_ref.connect("P1",rot1_ref["P1"])

#Jalur Bawah
bend3_ref.connect("P0",MZI1_ref["P2"])
rot2_ref.connect("P0",bend3_ref["P1"])
bend5_ref.connect("P0",rot2_ref["P1"])

#MZM output
MZI2_ref.connect("P0",bend5_ref["P1"])
bend6_ref.connect("P1",MZI2_ref["P3"])

#Beam Splitter, Grating Coupler
pbs_ref.connect("P0",bend6_ref["P0"])

#Grating Coupler Output 1
gc2_ref.rotate(180)

qst2_ref.x_min = pbs_ref.x_max + 10
qst2_ref.y_mid = pbs_ref.y_max + 5

route_gc2_1 = pf.parametric.route(port1=(pbs_ref, "P2"), port2=(qst2_ref, "P0"), radius=5)
bob_top.add_reference(route_gc2_1)
gc2_ref.connect("P0", qst2_ref["P1"])

#Grating Coupler Output 2
gc3_ref.rotate(180)

qst3_ref.x_min = pbs_ref.x_max + 10
qst3_ref.y_mid = pbs_ref.y_max - 30

route_gc3_1 = pf.parametric.route(port1=(pbs_ref, "P1"), port2=(qst3_ref, "P0"), radius=5)
bob_top.add_reference(route_gc3_1)
gc3_ref.connect("P0", qst3_ref["P1"])

terminal_width = 15

# Terminals
bob_top.add_terminal(pf.Terminal((12,0), pf.Rectangle(size=(terminal_width, terminal_width), 
                                                        center=(rot1_ref.x_mid, MZI1_ref.y_max + terminal_width/2 + 20))), 
                       "V_BASE", add_structure=True)

bob_top.add_terminal(pf.Terminal((12,0), pf.Rectangle(size=(terminal_width, terminal_width), 
                                                        center=(rot1_ref.x_mid, MZI1_ref.y_min - terminal_width/2 - 20))), 
                       "GND", add_structure=True)

pads = bob_top.terminals

# Electrical routing

# V_BASE
bob_top.add(
    pf.parametric.route_manhattan(
        terminal1=pads["V_BASE"],
        terminal2=(MZI1_ref, "VN", 0),
        direction1="y",
        direction2="x",
        width=10,
    ),
    pf.parametric.route_manhattan(
        terminal1=pads["V_BASE"],
        terminal2=(MZI2_ref, "VP", 0),
        direction1="y",
        direction2="x",
        width=10,
    ),
)

# GND
bob_top.add(
    pf.parametric.route_manhattan(
        terminal1=pads["GND"],
        terminal2=(MZI1_ref, "VP", 0),
        direction1="y",
        direction2="y",
        width=10,
        waypoints=[(MZI1_ref.x_min + terminal_width/2 + 0.425, MZI1_ref.y_min), 
                   (rot1_ref.x_mid, MZI1_ref.y_min),
                   ]
    ),
    pf.parametric.route_manhattan(
        terminal1=pads["GND"],
        terminal2=(MZI2_ref, "VN", 0),
        direction1="y",
        direction2="y",
        width=10,
        waypoints=[(MZI2_ref.x_max - terminal_width/2 - 0.425, MZI2_ref.y_min), 
                   (rot1_ref.x_mid, MZI2_ref.y_min),
                   ]
    ),
)

bob_top.add_model(pf.Tidy3DModel(), "Tidy3D")

viewer(bob_top)

Progress: 100% 


c:\Users\irfan\AppData\Local\Programs\Python\Python313\Lib\site-packages\photonforge\parametric_utils.py:168: RuntimeWarning: Skipped point (22.675, -20.85) from route
  c = component_func(*args, **kwargs)
c:\Users\irfan\AppData\Local\Programs\Python\Python313\Lib\site-packages\photonforge\parametric_utils.py:168: RuntimeWarning: Skipped point (303.025, -20.85) from route
  c = component_func(*args, **kwargs)


In [45]:
pf.tidy3d_plot(bob_top, plot_type="3d")